<div align="center">
    <h2>TEXAS COVID-19 Vaccination Progress</h2>
    <img src="https://images.unsplash.com/photo-1513023001678-6927b70dc4a0?ixlib=rb-1.2.1&ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&auto=format&fit=crop&w=1050&q=80"  width="500" height="200">
    <br><span>Photo by <a href="https://unsplash.com/@accrualbowtie?utm_source=unsplash&amp;utm_medium=referral&amp;utm_content=creditCopyText">Ryan Wallace</a> on <a href="https://unsplash.com/s/photos/texas-state-capitol?utm_source=unsplash&amp;utm_medium=referral&amp;utm_content=creditCopyText">Unsplash</a></span>

</div>
<br>
<div align="left"> 
This notebook presents and analyzes information about COVID-19 vaccine doses allocated and administered in Texas, USA. 
</div><br>

<div style="left">As of Monday, March 29, 2021, everyone age 16 and older is now eligible to receive a COVID-19 vaccine in Texas. <a href="https://www.dshs.state.tx.us/coronavirus/immunize/vaccine.aspx#eligible"> Source </a></div><br>

<div style="left">The COVID-19 vaccine supply has increased to the point that Texas DSHS will no longer allocate vaccine to providers each week. As of May 10, providers can order vaccine from DSHS on a daily basis. The weekly allocations in this data set reflect allocations that were made between Dec. 14, 2020 – May 3,2021.
<a href="https://www.dshs.state.tx.us/coronavirus/immunize/vaccineallocations.aspx"> Source </a></div><br>


<div style="background-color:#60cff7; font-size:120%;  font-family:sans-serif; text-align:center"><b>Table of Content</b></div>

* [Summary 📊](#1)
* [Demographic analysis](#2)
* [Details](#3)
* [Vaccine distribution vs population](#4)
* [Vaccine Doses Allocated](#5)
* [Weekly trend](#6)
* [Vaccine types and providers](#7)
* [Texas choropleth maps](#8) 

In [ ]:
import numpy as np
import pandas as pd
import os
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
DATE = '15-Jun-2021'
SHEET_LABEL = 'By County'
GENDER_SHEET = 'By Age, Gender, Race'
AGE_SHEET = 'By Age, Day'
df = pd.read_csv('/kaggle/input/texas-covid19-vaccine-allocation/weekly_allocations.csv',index_col=None)
sheet = pd.read_excel('/kaggle/input/texas-covid19-vaccine-allocation/COVID-19 Vaccine Data by County.xlsx', sheet_name=SHEET_LABEL)
gender = pd.read_excel('/kaggle/input/texas-covid19-vaccine-allocation/COVID-19 Vaccine Data by County.xlsx', sheet_name=GENDER_SHEET)
age = pd.read_excel('/kaggle/input/texas-covid19-vaccine-allocation/COVID-19 Vaccine Data by County.xlsx', sheet_name=AGE_SHEET)

sheet.rename(columns={
    'County Name':'county', 'Public Health Region (PHR)':'phr',
       'Vaccine Doses Administered':'administered',
       'People Vaccinated with at least One Dose':'atleast_one_dose', 'People Fully Vaccinated':'fully_vaccinated',
       'Population, 16+':'pop_16_plus', 'Population, 65+':'pop_65_plus',
       'Population, Phase 1A Healthcare Workers':'pop_ph1a_hc_workers',
       'Population, Phase 1A Long-term Care Residents':'pop_ph1a_ltc_residents',
        #'Population, Phase 1B Any Medical Condition' : 'pop_ph1b_medical_conditions'#
        'Population, 16-64\n Any Medical Condition' : 'pop_ph1b_medical_conditions',
        'Population, Education and Child Care Personnel' : 'pop_ph1b_edu_childcare_personnel'
        }, inplace=True)
gender.rename(columns= {'Gender':'gender',
                        'Age Group':'age_grp',
                        'Race/Ethnicity':'race_ethnicity',
                        'Doses Administered':'doses_administered',
                        'People Vaccinated with at least One Dose':'atleast_one_dose',
                        'People Fully Vaccinated':'fully_vaccinated'},
             inplace=True)

age.rename(columns= {'Age Group':'age_grp',
                     'Vaccination Date':'vacc_date',
                     'Doses Administered':'doses_administered'}, 
           inplace=True)

daily = sheet.copy()
tx_color = '#bf0a30'
print("Weekly data shape {} \n Daily data shape {} \nGender data shape {} \n   Age data shape {}".format(df.shape, sheet.shape, gender.shape, age.shape))

In [ ]:
daily.head()

In [ ]:
gender.rename(columns={'Gender ':'gender'}, inplace=True)
gender.head()

In [ ]:
#gender = gender.fillna(method='ffill') # to forward fill the gender column from excel pivot
#gender

In [ ]:
gender_df = gender.loc[gender['age_grp'] != 'Total']
gender_df

In [ ]:
daily.drop(daily.tail(1).index,inplace=True)

In [ ]:
daily = daily.drop(index=1).reset_index(drop=True)
daily = daily.drop(index=1).reset_index(drop=True)

In [ ]:
daily.info()

In [ ]:
#sns.heatmap(daily.isna())
daily['pop_16_plus'] = daily['pop_16_plus'].fillna(0)
daily['pop_65_plus'] = daily['pop_65_plus'].fillna(0)
daily['pop_ph1a_hc_workers'] = daily['pop_ph1a_hc_workers'].fillna(0)
daily['pop_ph1a_ltc_residents'] = daily['pop_ph1a_ltc_residents'].fillna(0)
daily['pop_ph1b_medical_conditions'] = daily['pop_ph1b_medical_conditions'].fillna(0)
daily['pop_ph1b_edu_childcare_personnel'] = daily['pop_ph1b_edu_childcare_personnel'].fillna(0)
#print(daily[daily['pop_65_plus'].isnull()])

In [ ]:
daily['administered'] = daily['administered'].astype(int)
daily['atleast_one_dose'] = daily['atleast_one_dose'].astype(int)
daily['fully_vaccinated'] = daily['fully_vaccinated'].astype(int)
daily['pop_16_plus'] = daily['pop_16_plus'].astype(int)
daily['pop_65_plus'] = daily['pop_65_plus'].astype(int)
daily['pop_ph1a_hc_workers'] = daily['pop_ph1a_hc_workers'].astype(int)
daily['pop_ph1a_ltc_residents'] = daily['pop_ph1a_ltc_residents'].astype(int)
daily['pop_ph1b_medical_conditions'] = daily['pop_ph1b_medical_conditions'].astype(int)
daily['pop_ph1b_edu_childcare_personnel'] = daily['pop_ph1b_edu_childcare_personnel'].astype(int)
daily.info()

In [ ]:
daily['total_percent_vaccd'] = round((daily['administered'] / daily['pop_16_plus'])*100,2)
daily.at[0,'county'] = '<b>Texas - Total</b>'

<a id='1'></a>
# <div style="background-color:#60cff7; font-size:120%; text-align:center">Summary</div>


In [ ]:
from plotly.subplots import make_subplots

total_data = None
total_data = {'16+':daily.at[0, 'pop_16_plus'],
              '65+':daily.at[0, 'pop_65_plus'],
              'Healthcare Workers':daily.at[0, 'pop_ph1a_hc_workers'],
              'Long-term Care Residents':daily.at[0, 'pop_ph1a_ltc_residents'],              
              'Any Medical Condition':daily.at[0, 'pop_ph1b_medical_conditions'],
              'School and Child Care Personnel':daily.at[0, 'pop_ph1b_edu_childcare_personnel'],              
              'Vaccine Doses Administered':daily.at[0, 'administered'],                            
              'Vaccinated with at least One Dose':daily.at[0, 'atleast_one_dose'],              
              'Fully Vaccinated':daily.at[0, 'fully_vaccinated']                            
             }

cols = {'16+':'pop_16_plus',
 '65+':'pop_65_plus',
 'Healthcare Workers':'pop_ph1a_hc_workers',
 'Long-term Care Residents':'pop_ph1a_ltc_residents',
 'Any Medical Condition':'pop_ph1b_medical_conditions',        
'School and Child Care Personnel':'pop_ph1b_edu_childcare_personnel',              
 'Vaccine Doses Administered':'administered',
 'Vaccinated with at least One Dose':'atleast_one_dose',
 'Fully Vaccinated':'fully_vaccinated'}

bar_data = []
for key, val in cols.items():
    bar_data.append(
        go.Bar(name=key, x=[key], y=[daily.at[0, val]],text=[daily.at[0, val]],
                  textposition='outside',
                   texttemplate='%{text:.2s}')
    )


total_df = pd.DataFrame.from_dict(total_data,orient='index', columns=['count']).reset_index()



fig = go.Figure(data=bar_data)
fig.update_layout(xaxis= { 'tickfont' : {'size':18},  "zeroline":False, "showgrid":False,
                         "fixedrange": False,                           "tickangle" : 45
                        }, 
                   title = { 'text' : '<b>Population vs Vaccinated - as of {}</b>'.format(DATE),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  #title_font_family = 'Computer Modern',
                 yaxis={'title':'Population','range':[0, 26000000],"zeroline":False, "showgrid":False,
                         "fixedrange": False},
                 font=dict(family='Computer Modern',size=15, color="#00040a"),
                 showlegend=False,
                plot_bgcolor="#f5f3e9",
                margin={"r":20, "l":30}
                 )


fig.add_annotation(
        x=2,
        y=23000000,
        xref="x",
        yref="y",
        text="Population Estimates are from <br> Texas Demographic Center",
        showarrow=False,
        font=dict(
            #family="Computer Modern",
            size=12,
            color="#fafafa"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#f58e5b",
        borderwidth=2,
        borderpad=4,
        bgcolor="#eb5409",
        opacity=0.8
        )

fig.add_annotation(
        x=7.5,
        y=20500000,
        xref="x",
        yref="y",
        text="Total vaccinated so far",
        showarrow=False,
        font=dict(
            #family="Courier New, monospace",
            size=12,
            color="#fafafa"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#9cc4f0",
        borderwidth=2,
        borderpad=4,
        bgcolor="#527ccc",
        opacity=0.8
        )

iplot(fig)

In [ ]:
tm = pd.read_excel('/kaggle/input/texas-covid19-vaccine-allocation/COVID-19 Vaccine Data by County.xlsx', sheet_name="By Vaccination Date")
tm.rename(columns={'Vaccination Date':'date',
                   'Doses Administered':'doses_administered',
                   'People Vaccinated with at least One Dose' : 'atleast_1_dose',
                   'People Fully Vaccinated ' : 'fully_vaccinated'}, inplace=True)
tm.drop(tm.tail(1).index,inplace=True)
tm['date'] = pd.to_datetime(tm['date'])
tm

In [ ]:
#Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=tm['date'], 
                         y=tm['atleast_1_dose'],
                         mode='lines+markers',
                         name = 'Vaccinated with atleast one dose',
                         line=dict(color='#bf0a30', width = 1.5,)
                        )
             )
              
fig.add_trace(go.Scatter(x=tm['date'], 
                         y=tm['fully_vaccinated'],
                         mode='lines+markers',
                         name = 'Fully Vaccinated',
                         line=dict(color='#03bafc', width = 1.5, )
                        )                            
              
             )
fig.update_layout(showlegend=True,
                   title = { 'text' : '<b>Vaccination Progress</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Vaccine Doses",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

In [ ]:
#Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=tm['date'], 
                         y=tm['doses_administered'],
                         mode='lines+markers',
                         name = 'Daily doses',
                         line=dict(color='#0003bf', width = 3.5)
                        )
             )

fig.update_layout(showlegend=True,
                   title = { 'text' : '<b>Daily vaccine doses</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Doses",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

In [ ]:
gender_age = gender_df.groupby(['gender', 'age_grp']).sum().reset_index()
gender_age = gender_age.loc[gender_age['gender'] != 'Texas']
gender_age

<a id='2'></a>
# <div style="background-color:#60cff7; font-size:120%; text-align:center">Demographic analysis</div>

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(name='Gender-Age',
                   z=gender_age['doses_administered'],
                   x=gender_age['gender'],
                   y=gender_age['age_grp'],
                   hoverongaps = True,
                   colorscale = 'agsunset_r',
                    hovertemplate = 'Age group : %{y}<br>Gender : %{x}<br><b>Doses :%{z}</b>'))
fig.update_layout(xaxis= { 'tickfont' : {'size':18},  "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                                                    "tickangle" : 45
                        }, 
                   title = { 'text' : '<b>Vaccinations by gender and age as of {}</b>'.format(DATE),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  #title_font_family = 'Computer Modern',
                 yaxis={'title':'Age group',"zeroline":False, "showgrid":False,
                         "fixedrange": False},
                 font=dict(family='Computer Modern',size=15, color="#00040a"),
                 showlegend=True,
                plot_bgcolor="#f5f3e9",
                margin={"r":20, "l":30}
                 )
iplot(fig)

In [ ]:
gender_race = gender_df.groupby(['gender', 'race_ethnicity']).sum().reset_index()
gender_race = gender_race.loc[gender_race['gender'] != 'Texas']
gender_race

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(name='Race-Gender',
                   z=gender_race['doses_administered'],
                   x=gender_race['gender'],
                   y=gender_race['race_ethnicity'],
                   hoverongaps = True,
                   colorscale = 'agsunset_r',
                    hovertemplate = 'Race : %{y}<br>Gender : %{x}<br><b>Doses :%{z}</b>'))

fig.update_layout(xaxis= { 'tickfont' : {'size':18},  "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                                                    "tickangle" : 45
                        }, 
                   title = { 'text' : '<b>Vaccinations by gender and ethnicity as of {}</b>'.format(DATE),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  #title_font_family = 'Computer Modern',
                 yaxis={'title':'Race',"zeroline":False, "showgrid":False,
                         "fixedrange": False},
                 font=dict(family='Computer Modern',size=15, color="#00040a"),
                 showlegend=True,
                plot_bgcolor="#f5f3e9",
                margin={"r":20, "l":30}
                 )
iplot(fig)

In [ ]:
age_race = gender_df.groupby(['age_grp', 'race_ethnicity']).sum().reset_index()
age_race = age_race.loc[age_race['race_ethnicity'] != 'Total']
age_race

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(name='Age-Race',
                   z=age_race['doses_administered'],
                   x=age_race['race_ethnicity'],
                   y=age_race['age_grp'],
                   hoverongaps = True,
                   colorscale = 'agsunset_r',
                    hovertemplate = 'Age group : %{y}<br>Race : %{x}<br><b>Doses :%{z}</b>'))
fig.update_layout(xaxis= { 'title':'Race','tickfont' : {'size':18},  "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                                                    "tickangle" : 45
                        }, 
                   title = { 'text' : '<b>Vaccinations by age and ethnicity as of {}</b>'.format(DATE),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  #title_font_family = 'Computer Modern',
                 yaxis={'title':'Age group',"zeroline":False, "showgrid":False,
                         "fixedrange": False},
                 font=dict(family='Computer Modern',size=15, color="#00040a"),
                 showlegend=True,
                plot_bgcolor="#f5f3e9",
                margin={"r":20, "l":30}
                 )
iplot(fig)

In [ ]:
age_groups = list(age['age_grp'].unique())
age_groups.remove('Grand Total')
age_groups.remove('Unknown')

#Plot
fig = go.Figure()
for grp in age_groups:
    age_grp_df = age.loc[age['age_grp'] == grp].reset_index(drop=True)
    fig.add_trace(go.Scatter(x=age_grp_df['vacc_date'], 
                             y=age_grp_df['doses_administered'],
                             mode='lines+markers',
                             name = grp,
                             #line=dict(color='#54e871', width = 3)
                            )
                 )

fig.update_layout(showlegend=True,
                   title = { 'text' : '<b>Doses administered by age - Time series</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Doses administered",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

<a id='3'></a>
# <div style="background-color:#60cff7; font-size:120%; text-align:center">Details</div>


In [ ]:
#daily.head()
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

color = [[rowOddColor,rowEvenColor,rowOddColor,rowEvenColor]*len(daily.index)]

fig = go.Figure(data=[go.Table(
    columnwidth = [1,1,1.25,1.25,1.25],
    header=dict(values=['<b>County</b>',
                        '<b>Doses <br>Administered</b>', 
                        '<b>People <br>received <br>atleast one dose</b>', 
                        '<b>People <br>Fully <br>vaccinated</b>',
                       '<b>% of population <br> received <br>atleast one dose</b>'],
                fill_color=tx_color,
                font=dict(color='white',size=12),
                line_color='darkslategray',
                align=['left','center']),
    cells=dict(values=[daily['county'], 
                       daily['administered'], 
                       daily['atleast_one_dose'], 
                       daily['fully_vaccinated'],
                       daily['total_percent_vaccd']],
               fill_color=color,
               line_color='darkslategray',
               align=['left','center'],
              font=dict(color='darkslategray',size=13)
              )
    )
])
fig.update_layout(title=dict(text='<b>Vaccine doses administered per county <br> as of {}</b>'.format(DATE),
                                                   x=0.5,
                                                   font_color='black'),
                  margin=dict(r=20, l=20)
                 )
                                                
                  
iplot(fig)

In [ ]:
TOP_COUNT = 20
top_counties = daily.sort_values(['administered','total_percent_vaccd'],ascending = False).head(TOP_COUNT).reset_index()
top_counties.drop(daily.head(1).index,inplace=True)

In [ ]:
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

color = [[rowOddColor,rowEvenColor,rowOddColor,rowEvenColor]*len(daily.index)]

fig = go.Figure(data=[go.Table(
    columnwidth = [1,1,1.25,1.25,1.25],
    header=dict(values=['<b>County</b>',
                        '<b>Doses <br>Administered</b>', 
                        '<b>People <br>received <br>atleast one dose</b>', 
                        '<b>People <br>Fully <br>vaccinated</b>',
                       '<b>% of population <br> received <br>atleast one dose</b>'],
                fill_color=tx_color,
                font=dict(color='white',size=12),
                line_color='darkslategray',
                align=['left','center']),
    cells=dict(values=[top_counties['county'], 
                       top_counties['administered'], 
                       top_counties['atleast_one_dose'], 
                       top_counties['fully_vaccinated'],
                       top_counties['total_percent_vaccd']],
               fill_color=color,
               line_color='darkslategray',
               align=['left','center'],
              font=dict(color='darkslategray',size=13)
              )
    )
])
fig.update_layout(title=dict(text='<b>Top {} counties with most administered doses <br> as of {}</b>'.format(TOP_COUNT,DATE),
                                                   x=0.5,
                                                   font_color='black'),
                  margin=dict(r=20, l=20)
                 )
                                                
                  
iplot(fig)

In [ ]:

fig = go.Figure([go.Bar(x=top_counties['county'], 
                        y=top_counties['administered'],
                        text = top_counties['administered'],
                       textposition='outside',
                       texttemplate='%{text:.2s}')])

fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Top {} counties with most administered doses <br>as of {}</b>'.format(TOP_COUNT,DATE),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses administered",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                           "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
fig.show()

In [ ]:
top_counties = daily.sort_values(['total_percent_vaccd'],ascending = False).head(TOP_COUNT).reset_index()
top_counties.drop(daily.head(1).index,inplace=True)

fig = go.Figure([go.Bar(x=top_counties['county'], 
                        y=top_counties['total_percent_vaccd'],
                        text = top_counties['total_percent_vaccd'],
                       textposition='outside',
                       texttemplate='%{text:}%')])

fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Top {} counties with most % of population vaccinated <br>as of {}</b>'.format(TOP_COUNT,DATE),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "% of population",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                           "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

> I believe the counties showing more than 100% of population vaccinated indicates that people from neighboring counties traveling into nearby counties to get the vaccine.

In [ ]:
top_counties = daily.sort_values(['total_percent_vaccd'],ascending = True).head(TOP_COUNT).reset_index()
top_counties.drop(daily.head(1).index,inplace=True)

fig = go.Figure([go.Bar(x=top_counties['county'], 
                        y=top_counties['total_percent_vaccd'],
                        text = top_counties['total_percent_vaccd'],
                       textposition='outside',
                       texttemplate='%{text:}%')])

fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Bottom {} counties with least % of population vaccinated <br>as of {}</b>'.format(TOP_COUNT,DATE),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "% of population",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                           "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

<a id='4'></a>
# <div style="background-color:#60cff7; font-size:120%; text-align:center">Vaccine distribution vs population</div>

The following plots show that the vaccine distribution is proportional to the population across counties

In [ ]:
import plotly.figure_factory as ff
import numpy as np

daily = daily.loc[daily['county'] != 'Other']
# histogram data
hist_data = [np.log10(daily['administered']), 
             np.log10(daily['pop_65_plus']+daily['pop_ph1a_hc_workers']+daily['pop_ph1a_ltc_residents']),
             np.log10(daily['pop_ph1b_medical_conditions']),
             np.log10(daily['pop_16_plus']),             
            ]

group_labels = ['Doses administered', 'Phase Ia population to be vaccinated','Phase Ib population to be vaccinated','Total population to be vaccinated']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, 
                         group_labels, 
                         bin_size=.2,
                        #curve_type='normal',
                        colors=[tx_color,'#34a4eb', '#1317f0','#303328'],
                        show_rug=False,
                    )
fig.update_layout(showlegend=True,
                  title = { 'text' : '<b>Frequency distribution of vaccine doses vs Population<b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Population in log scale",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)


In [ ]:
from sklearn.linear_model import LinearRegression
counties = daily.copy()
counties.drop(counties.head(1).index,inplace=True)
X = counties['pop_65_plus'].values.reshape(-1,1)

X = counties['pop_16_plus'].values.reshape(-1,1)
model = LinearRegression()
model.fit(X, counties['administered'])

x_range = np.linspace(X.min(), X.max(), 100)
y_range = model.predict(x_range.reshape(-1,1))

fig = go.Figure(data=go.Scatter(
    x=counties['pop_16_plus'],
    y=counties['administered'],
    mode='markers',
    marker=dict(size=counties['administered']/18000, color = ['#2100f5']*len(counties.county)),
    hovertemplate =
    '<b>%{text}</b><br>' + 
    '<i>Population</i>: %{x}'+
    '<br><b>Doses administered</b>: %{y}<br>',
    text = ['{}'.format(i) for i in counties['county']],
))

fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Regression Fit',marker=dict(size=2.5,
                color='#13111f')))

fig.update_layout(showlegend=False,
                  title = { 'text' : '<b>Regression Trend - County popluation vs Doses administered</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Number of doses administered",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={"title": "County Population",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

<a id='5'></a>
# <div style="background-color:#60cff7; font-size:120%; text-align:center">Vaccine Doses Allocated</div>
This section presents the allocated vaccine doses across counties based on the weekly report. So far there are 11 weekly reports published by Texas Department of State Health Services starting from 12-Dec-2020

In [ ]:
total = df['allocated_doses'].sum()
week4_federal_pharmacy = 121875 # This list does not include 121,875 Pfizer doses allocated to the federal pharmacy/long-term care partnership.(as mentioned in dshs)
week5_federal_pharmacy = 121875
total += week4_federal_pharmacy
total += week5_federal_pharmacy
print(f"{total:,} doses allocated")

## Type of vaccine allocated

Vaccines are allocated in batches from producers (Moderna, Pfizer). Some providers are allocated with combination of Moderna and Pfizer so exact count of those allocations are not reported.

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2,
                   specs=[[{"type": "bar"},{"type": "pie"}]],
                    column_widths=[0.4, 0.6],
                   subplot_titles=["Vaccines allocated since 12-Dec-2020 till today", "% of allocation"])

df_bar = df.groupby(['vaccine_type'])['allocated_doses'].sum().to_frame().reset_index()
df_bar.at[2, 'allocated_doses'] = df_bar.at[2, 'allocated_doses'] + week4_federal_pharmacy
df_bar.at[0,'color'] = '#00040a'
df_bar.at[1,'color'] = '#cc34eb'
df_bar.at[2,'color'] = '#bf0a30'
df_bar.at[3,'color'] = '#03bafc'
df_bar.at[0,'vaccine_type'] = 'Combination (Pfizer + Moderna)'
df_bar.at[1,'vaccine_type'] = 'Johnson & Johnson'
df_bar['percent'] = 0.0
df_bar['percent'] = round((df_bar['allocated_doses'] / df_bar['allocated_doses'].sum()) * 100, 2)

fig.add_trace(
                go.Bar(name=df_bar.columns.values[0], 
                        x=df_bar['vaccine_type'], 
                        y=df_bar['allocated_doses'], 
                        text=df_bar['allocated_doses'], 
                        marker=dict(color=df_bar['color']),
                        textposition='outside',
                       texttemplate='%{text:.2s}'
                      ),
                row=1, col=1
            )


fig.add_trace(
                go.Pie(labels=df_bar['vaccine_type'], 
                        values=df_bar['percent'],
                        marker=dict(colors=df_bar['color']),                       
                        name="vaccine type percent",
                       textinfo='label+percent',
                       hoverinfo='label+percent',                       
                       insidetextorientation='radial',
                       pull=[0.05,0.05,0.05]
                      ),
                row=1, col=2)



fig.update_layout(xaxis=dict(categoryorder='total descending',tickfont=dict(size=18)), 
                  title_font_family = 'Computer Modern',
                 yaxis={'title':'Number of Doses','range':[0,6500000]},
                 font=dict(family='Computer Modern',size=15, color="#00040a"),
                 showlegend=False)

fig.update_layout(showlegend=False,
                  title_font_size=20,
                  title_font_color='black',
                  yaxis={"title": "Number of doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                         "tickangle" : -45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

<a id='6'></a>
# <div style="background-color:#60cff7; font-size:120%; text-align:center">Weekly trend</div>

> Let's look at the weekly vaccine progress in Texas and plot the trendline to look at how Texas is progressing vs rest of USA

In [ ]:
#Data
weekly_trend = df.groupby(['week_starting'])['allocated_doses'].sum().reset_index()
weekly = df.groupby(['week_starting','vaccine_type'])['allocated_doses'].sum().reset_index()
moderna = weekly.loc[weekly['vaccine_type']=='Moderna']
pfizer = weekly.loc[weekly['vaccine_type']=='Pfizer']
johnson = weekly.loc[weekly['vaccine_type']=='Johnson']
combination = weekly.loc[weekly['vaccine_type']=='Combination']

#Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=weekly_trend['week_starting'], 
                         y=weekly_trend['allocated_doses'],
                         mode='lines+markers',
                         name = 'Weekly total allocation',
                         line=dict(color='#0003bf', width = 3.5)
                        )
             )

fig.add_trace(go.Scatter(x=moderna['week_starting'], 
                         y=moderna['allocated_doses'],
                         mode='lines+markers',
                         name = 'Moderna',
                         line=dict(color='#bf0a30', width = 1.5,dash='dot')
                        )
             )
              
fig.add_trace(go.Scatter(x=pfizer['week_starting'], 
                         y=pfizer['allocated_doses'],
                         mode='lines+markers',
                         name = 'Pfizer',
                         line=dict(color='#03bafc', width = 1.5, dash='dot')
                        )                            
              
             )
fig.add_trace(go.Scatter(x=combination['week_starting'], 
                         y=combination['allocated_doses'],
                         mode='lines+markers',
                         name = 'Combination <br> (Moderna + Pfizer)',
                         line=dict(color='#00040a', width = 1.5, dash='dot')
                        )                            
              
             )
fig.add_trace(go.Scatter(x=johnson['week_starting'], 
                         y=johnson['allocated_doses'],
                         mode='lines+markers',
                         name = 'Johnson & Johnson',
                         line=dict(color='#cc34eb', width = 1.5, dash='dot')
                        )                            
              
             )

fig.update_layout(showlegend=True,
                   title = { 'text' : '<b>Weekly doses allocation trend for Texas</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

> No new allocations of Johnson & Johnson vaccines since the week of 12-Apr-2021

In [ ]:
#Data
country = pd.read_csv('/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv',index_col=None)
usa = country.loc[country['iso_code'] == "USA"]
usa = usa[['country','iso_code','date','total_vaccinations']].reset_index(drop=True)
usa.fillna(0)
weekly_trend['cumulative'] = weekly_trend['allocated_doses'].cumsum()

#Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=usa['date'], 
                         y=np.log(usa['total_vaccinations']),
                         mode='lines+markers',
                         name = 'USA',
                         line=dict(color='#0003bf', width = 3.5)
                        )
             )
fig.add_trace(go.Scatter(x=weekly_trend['week_starting'], 
                         y=np.log(weekly_trend['cumulative']),
                         mode='lines+markers',
                         name = 'Texas',
                         line=dict(color='#bf0a30', width = 3.5)
                        )
             )
fig.update_layout(showlegend=True,
                   title = { 'text' : '<b>Vaccination progress trend - USA vs Texas</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Doses in log scale",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)


In [ ]:
#Data

top_10_counties = list(df.groupby(['county'])['allocated_doses'].sum().reset_index().sort_values('allocated_doses',ascending=False).reset_index(drop=True).head(10)['county'])
weekly_county_trend = df.groupby(['week_starting','county'])['allocated_doses'].sum().reset_index().sort_values('allocated_doses',ascending=False).reset_index(drop=True)

#Plot
fig = go.Figure()
for county in top_10_counties:
    county_trend_df = weekly_county_trend.loc[weekly_county_trend['county'] == county].sort_values('week_starting').reset_index(drop=True)
    fig.add_trace(go.Scatter(x=county_trend_df['week_starting'], 
                             y=county_trend_df['allocated_doses'],
                             mode='lines+markers',
                             name = county,
                             #line=dict(color='#54e871', width = 3)
                            )
                 )

fig.update_layout(showlegend=True,
                   title = { 'text' : '<b>Weekly allocation trend for top 10 counties</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

In [ ]:

top_10_cities = list(df.groupby(['city'])['allocated_doses'].sum().reset_index().sort_values('allocated_doses',ascending=False).reset_index(drop=True).head(10)['city'])
weekly_city_trend = df.groupby(['week_starting','city'])['allocated_doses'].sum().reset_index().sort_values('allocated_doses',ascending=False).reset_index(drop=True)
#Data


#Plot
fig = go.Figure()
for city in top_10_cities:
    city_trend_df = weekly_city_trend.loc[weekly_city_trend['city'] == city].sort_values('week_starting').reset_index(drop=True)
    fig.add_trace(go.Scatter(x=city_trend_df['week_starting'], 
                             y=city_trend_df['allocated_doses'],
                             mode='lines+markers',
                             name = city,
                             #line=dict(color='#54e871', width = 3)
                            )
                 )

fig.update_layout(showlegend=True,
                   title = { 'text' : '<b>Weekly allocation trend for top 10 cities</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

In [ ]:
rowEvenColor = 'lightgrey'
rowOddColor = 'white'
citywise = df.groupby(['city'])['allocated_doses'].sum().reset_index()

color = [[rowOddColor,rowEvenColor]*len(citywise.index)]

fig = go.Figure(data=[go.Table(
    columnwidth = [1,1,1.25,1.25,1.25],
    header=dict(values=['<b>City</b>',
                        '<b>Doses Allocated</b>', 
                        ],
                fill_color=tx_color,
                font=dict(color='white',size=12),
                line_color='darkslategray',
                align=['left','center']),
    cells=dict(values=[citywise['city'], 
                       citywise['allocated_doses'], 
                        ],
               fill_color=color,
               line_color='darkslategray',
               align=['left','center'],
              font=dict(color='darkslategray',size=13)
              )
    )
])
fig.update_layout(title=dict(text='<b>Cummulative weekly allocations per city <br>since 12-Dec-2020</b>',
                                                   x=0.5,
                                                   font_color='black'),
                  margin=dict(r=20, l=20)
                 )
                                                
                  
fig.show()

In [ ]:
pfizer = df.loc[df['vaccine_type'].isin(['Pfizer'])].groupby('county')['allocated_doses'].sum().sort_values(ascending=False).reset_index()
pfizer

<a id='7'></a>
# <div style="background-color:#60cff7; font-size:120%; text-align:center">Vaccine types and providers</div>

In [ ]:
#df_bar.at[0,'color'] = '#00040a'
#df_bar.at[1,'color'] = '#bf0a30'
#df_bar.at[2,'color'] = '#03bafc
fig = go.Figure([go.Bar(x=pfizer['county'], 
                        y=pfizer['allocated_doses'],
                        text = pfizer['allocated_doses'],
                        marker=dict(color='#03bafc'),
                       textposition='outside',
                       texttemplate='%{text:.2s}')])

fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Counties that received Pfizer vaccine</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                          "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

In [ ]:
pfizer = df.loc[df['vaccine_type'].isin(['Combination'])].groupby('county')['allocated_doses'].sum().sort_values(ascending=False).reset_index()
pfizer

In [ ]:
fig = go.Figure([go.Bar(x=pfizer['county'], 
                        y=pfizer['allocated_doses'],
                        text = pfizer['allocated_doses'],
                        marker=dict(color='#88fc03'),
                       textposition='outside',
                       texttemplate='%{text:.2s}')])

fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Counties that received Pfizer + Moderna combination of doses</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                          "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

> Other counties received only Moderna vaccine doses. 

In [ ]:
top_cities = citywise.sort_values('allocated_doses',ascending=False).head(10)

fig = go.Figure([go.Bar(x=top_cities['city'], 
                        y=top_cities['allocated_doses'],
                        text = top_cities['allocated_doses'],
                       textposition='outside',
                       texttemplate='%{text:.2s}')])

fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Top 10 cities with most allocated vaccine doses</b>',
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                          "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
iplot(fig)

### Vaccine Providers

In [ ]:
TOP_PROVIDERS = 40
provider_df = df.groupby(['vaccine_type','provider_name','city']).sum().reset_index()
provider_df = provider_df.sort_values('allocated_doses',ascending=False).head(TOP_PROVIDERS).reset_index()
#pfizer_provider = provider_df.loc[provider_df['vaccine_type'] == 'Moderna'].sort_values('allocated_doses',ascending=False)
#pfizer_provider.head()
provider_df

In [ ]:
print("Top {} provider doses {}\nTotal doses {}\nPercentage {}".format(TOP_PROVIDERS,
                                                                      provider_df['allocated_doses'].sum(),
                                                                  df['allocated_doses'].sum(),
                                                                  provider_df['allocated_doses'].sum()/df['allocated_doses'].sum()*100))



In [ ]:
#df.loc[(df['vaccine_type'] == 'Pfizer') & (df['city']=='Austin')]['allocated_doses'].sum()
import plotly.express as px
fig = px.treemap(provider_df, 
                 path=['vaccine_type','city','provider_name'], 
                 values='allocated_doses',
                color='allocated_doses',
                color_continuous_scale='RdBu',
                color_continuous_midpoint=np.mean(provider_df['allocated_doses']),
                )
fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Top {} vaccine providers across Texas</b>'.format(TOP_PROVIDERS),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                          "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
fig.show()

In [ ]:
provider_df = df.groupby(['vaccine_type','provider_name','city']).sum().reset_index()
provider_df = provider_df.sort_values('allocated_doses',ascending=False).reset_index(drop=True)
pfizer_provider = provider_df.loc[provider_df['vaccine_type'] == 'Pfizer'].sort_values('allocated_doses',ascending=False)
pfizer_provider = pfizer_provider.sort_values('allocated_doses',ascending=False).head(TOP_PROVIDERS).reset_index()
pfizer_provider

In [ ]:
#df.loc[(df['vaccine_type'] == 'Pfizer') & (df['city']=='Austin')]['allocated_doses'].sum()
import plotly.express as px
fig = px.treemap(pfizer_provider, 
                 path=['vaccine_type','city','provider_name'], 
                 values='allocated_doses',
                color='allocated_doses',
                color_continuous_scale='deep',
                color_continuous_midpoint=np.mean(pfizer_provider['allocated_doses']),
                )
fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>All Pfizer vaccine providers</b>'.format(TOP_PROVIDERS),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                          "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
fig.show()

In [ ]:
provider_df = df.groupby(['vaccine_type','provider_name','city']).sum().reset_index()
provider_df = provider_df.sort_values('allocated_doses',ascending=False).reset_index(drop=True)
moderna_provider = provider_df.loc[provider_df['vaccine_type'] == 'Moderna'].sort_values('allocated_doses',ascending=False)
moderna_provider_top = moderna_provider.sort_values('allocated_doses',ascending=False).head(TOP_PROVIDERS).reset_index()
moderna_provider_top

In [ ]:
#df.loc[(df['vaccine_type'] == 'Pfizer') & (df['city']=='Austin')]['allocated_doses'].sum()
import plotly.express as px
fig = px.treemap(moderna_provider_top, 
                 path=['vaccine_type','city','provider_name'], 
                 values='allocated_doses',
                color='allocated_doses',
                color_continuous_scale='matter',
                color_continuous_midpoint=np.average(moderna_provider_top['allocated_doses']),
                )
fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Top {} Moderna vaccine providers</b>'.format(TOP_PROVIDERS),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                          "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
fig.show()

In [ ]:
provider_df = df.groupby(['vaccine_type','provider_name','city']).sum().reset_index()
provider_df = provider_df.sort_values('allocated_doses',ascending=False).reset_index(drop=True)
johnson_provider = provider_df.loc[provider_df['vaccine_type'] == 'Johnson'].sort_values('allocated_doses',ascending=False)
johnson_provider_top = johnson_provider.sort_values('allocated_doses',ascending=False).head(TOP_PROVIDERS).reset_index()
fig = px.treemap(johnson_provider_top, 
                 path=['vaccine_type','city','provider_name'], 
                 values='allocated_doses',
                color='allocated_doses',
                color_continuous_scale='purp',
                color_continuous_midpoint=np.average(johnson_provider_top['allocated_doses']),
                )
fig.update_layout(showlegend=False,
                   title = { 'text' : '<b>Top {} Johnson & Johnson vaccine providers</b>'.format(TOP_PROVIDERS),
                          'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        },
                  title_font_color='black',
                  yaxis={"title": "Total doses allocated",
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False
                        },
                   xaxis={
                         "zeroline":False, "showgrid":False,
                         "fixedrange": False,
                          "tickangle" : 45
                        },
                  plot_bgcolor="#f5f3e9",
                  margin={"r":20, "l":30},
                 )
fig.show()

<a id='8'></a>
# <div style="background-color:#60cff7; font-size:120%; text-align:center">Texas choropleth maps</div>

In [ ]:
import bokeh
bokeh.sampledata.download()

In [ ]:
from bokeh.io import show
from bokeh.models import LogColorMapper, LinearColorMapper
from bokeh.palettes import Viridis256 as palette
from bokeh.plotting import figure
from bokeh.sampledata.us_counties import data as counties
from bokeh.io import output_notebook
#from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.models import ColorBar, BasicTicker, FixedTicker, FuncTickFormatter, PrintfTickFormatter
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6

output_notebook()
palette = tuple(reversed(palette))

#mapper = linear_cmap(field_name='administered', palette=Spectral6 ,low=min(daily['administered']) ,high=max(daily['administered']))

counties = {
    code: county for code, county in counties.items() if county["state"] == "tx"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]
county_rates = daily[daily['county'].isin(county_names)]['administered']
#color_mapper = LogColorMapper(palette=palette)
color_mapper = LinearColorMapper(palette=palette, low=min(daily['administered']), high=np.mean(daily['administered']))

color_bar = ColorBar(color_mapper=color_mapper, major_label_text_font_size="10px",
                     ticker=BasicTicker(desired_num_ticks=15, base=10, mantissas=[1, 2,3,4,5,6,7,8]),
                     label_standoff=6, border_line_color=None, location=(0, 0), orientation='horizontal')

#color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=550, height=20,
#                         location=(0,0), orientation='horizontal')

data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
)

TOOLS = "pan,wheel_zoom,reset,hover,save"
p = figure(
    title="Texas COVID-19 vaccine doses administered as of {}".format(DATE), tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("Total doses administered", "@rate"), ("(Long, Lat)", "($x, $y)")
    ]
    )
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"
p.title.text_font_size = '12pt'
p.title.text_font = 'Computer Modern'

p.patches('x', 'y', source=data,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)
p.add_layout(color_bar,'below')

show(p)


<div align="left">
    <h2>Texas Vaccine Rollout Timeline</h2>
<span>
To date, Texas has opened up vaccine eligibility according to the following timeline:
<ul>
    <li> May 12, 2021: Everyone 12 years old and older
    <li> March 29, 2021: Everyone 16 years old and older
    <li> March 15, 2021: Phase 1C (people 50 to 64 years of age)
    <li> March 3, 2021: Schools and licensed child care personnel
    <li> December 29, 2020: Phase 1B (people 65+ or people 16+ with a health condition that increases risk of severe COVID‑19 illness)
    <li> December 14, 2020: Phase 1A (front-line healthcare workers and residents at long-term care facilities)
</ul>
Texas continues to receive doses of the Pfizer, Moderna, and Johnson & Johnson COVID-19 vaccines, and is distributing statewide to hospitals, pharmacies, local health departments, freestanding ERs, and other clinics</span><br>
<div>
Source : <a href="https://www.dshs.state.tx.us/coronavirus/immunize/vaccine.aspx#eligible">Texas Department of State Health Services</a>
    </div>
 </div>

In [ ]:
#'pop_65_plus','pop_ph1a_hc_workers','pop_ph1a_ltc_residents','pop_ph1b_medical_conditions'
daily['phase1_pop'] = daily['pop_65_plus']+ daily['pop_ph1a_hc_workers']+ daily['pop_ph1a_ltc_residents']+ daily['pop_ph1b_medical_conditions']
daily['phase1_percent_vaccd'] = round((daily['administered'] / daily['phase1_pop'])*100,2)
#daily['pop_65_plus']+ daily['pop_ph1a_hc_workers']+ daily['pop_ph1a_ltc_residents']+ daily['pop_ph1b_medical_conditions']
daily.head()

In [ ]:
from bokeh.io import show
from bokeh.models import LogColorMapper, LinearColorMapper
#from bokeh.palettes import Viridis256 as palette
from bokeh.plotting import figure
from bokeh.sampledata.us_counties import data as counties
from bokeh.io import output_notebook
#from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.models import ColorBar, BasicTicker, FixedTicker, FuncTickFormatter, PrintfTickFormatter
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6 as palette

output_notebook()
palette = tuple(reversed(palette))

#mapper = linear_cmap(field_name='administered', palette=Spectral6 ,low=min(daily['administered']) ,high=max(daily['administered']))

counties = {
    code: county for code, county in counties.items() if county["state"] == "tx"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]
county_rates = daily[daily['county'].isin(county_names)]['phase1_percent_vaccd']
#color_mapper = LogColorMapper(palette=palette)
color_mapper = LinearColorMapper(palette=palette)

color_bar = ColorBar(color_mapper=color_mapper, major_label_text_font_size="10px",
                     #ticker=BasicTicker(desired_num_ticks=15, base=10, mantissas=[1, 2,3,4,5,6,7,8]),
                     label_standoff=6, border_line_color=None, location=(0, 0), orientation='horizontal')

#color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=550, height=20,
#                         location=(0,0), orientation='horizontal')

data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
)

TOOLS = "pan,wheel_zoom,reset,hover,save"
p = figure(
    title="Phase I - % of population received atleast one dose as of {}".format(DATE), tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("% of population to receive atleast one dose", "@rate"), ("(Long, Lat)", "($x, $y)")
    ]
    )
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"
p.title.text_font_size = '12pt'
p.title.text_font = 'Computer Modern'

p.patches('x', 'y', source=data,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)
p.add_layout(color_bar,'below')

show(p)

> Thanks for going through the notebook. I welcome any comments/feedback to improve further. 